# Исследование надежности заемщиков.

## Обзор данных

In [1]:
import pandas as pd
data = pd.read_csv('\datasets\data.csv')
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


## Предобработка данных

### Пропуски значений

In [2]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах `days_employed` и `total_income` обнаружены пропуски, которые составляют 10% от количества строк. Скорее всего, данные по случайности не были внесены составителями изначально. В наших данных могут быть значения, которые сильно выделяются среди большинства, поэтому заполнять пропуски средним будет некорректно. Заполним их медианным значением.

In [3]:
sorted_days_employed = data.sort_values(by='days_employed')
days_emp_median = sorted_days_employed['days_employed'].median()
# медиана по общему трудовому стажу

In [4]:
sorted_total_income = data.sort_values(by='total_income')
total_inc_median = sorted_total_income ['total_income'].median()
# медиана ежемесячного дохода

In [5]:
data['days_employed'] = data['days_employed'].fillna(days_emp_median)
data['total_income'] = data['total_income'].fillna(total_inc_median)
# заполнение пропусков

In [6]:
data.isna().sum() # проверка

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Проверка данных на аномалии и исправления

In [7]:
columns = data.columns
for c in columns:
    print(c.upper(), '\n', data[c].unique(), '\n')
# вывод уникальных значений в столбцах для проверки данных

CHILDREN 
 [ 1  0  3  2 -1  4 20  5] 

DAYS_EMPLOYED 
 [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853] 

DOB_YEARS 
 [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75] 

EDUCATION 
 ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень'] 

EDUCATION_ID 
 [0 1 2 3 4] 

FAMILY_STATUS 
 ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем'] 

FAMILY_STATUS_ID 
 [0 1 2 3 4] 

GENDER 
 ['F' 'M' 'XNA'] 

INCOME_TYPE 
 ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете'] 

DEBT 
 [0 1] 

TOTAL_INCOME 
 [253875.6394526  112080.01410244 145885.95229686 ...  89672.56115303
 244

Видим несколько проблем:
1. в столбце `children` значения -1 и 20;
2. в стоблце `days_employed` отрицательные значения;
3. в столбце `dob_years` значение 0;
4. в столбце `education` дубликаты;
5. столбцы `purpose` и `total_income` следует для удобства разбить на категории.

Сначала исправим аномалии.

In [8]:
data.loc[data['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,-601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,-1203.369529,53,среднее,1,женат / замужем,0,M,компаньон,0,145017.937533,покупка жилой недвижимости
21404,20,-494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


По статистике в России 23 750 000 семей с детьми, из которых в 943 более 11 детей. Я бы уточнила, ошибка в нашей выборке или нет. Пока примем за ошибку (в 21 год вероятность иметь столько детей очень мала) и уберем ноль. 

In [9]:
data.loc[(data.children == 20), 'children'] = 2
data.loc[(data.children == -1), 'children'] = 1
# исправляем значения

In [10]:
data['days_employed'] = abs(data['days_employed'])
# убираем отрицательные значения в стаже работы

In [11]:
sorted_age = data.sort_values(by='dob_years')
age_median = sorted_age ['dob_years'].median()
data.loc[(data.dob_years == 0), 'dob_years'] = int(age_median)
# заполняем значение 0 в возрасте клиента медианным значением по выборке

### Удаление дубликатов

In [12]:
data.duplicated().sum()
# подсчёт явных дубликатов
print(f"Кол-во явных дубликатов:{data.duplicated().sum()}")

Кол-во явных дубликатов:55


In [13]:
data = data.drop_duplicates().reset_index(drop=True) 
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
print(f"Кол-во явных дубликатов:{data.duplicated().sum()}") # проверка на отсутствие явных дубликатов

Кол-во явных дубликатов:0


In [14]:
data['education'] = data['education'].str.lower()
# обработка неявных дубликатов в столбце образование
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

### Категоризация целей кредита

Лучше всего токенизировать и лемматизировать строки, чтобы именно по леммам категоризировать цели кредита.

In [15]:
import pip
pip.main(['install', 'pymorphy2'])
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
# импортируем пайморфи

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Loading dictionaries from C:\Users\js\anaconda3\lib\site-packages\pymorphy2_dicts_ru\data
format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [16]:
def purpose_column(purpose):
    
    '''функция принимает на вход цели кредита, 
    токенизирует, лемматизирует,
    по лемме категоризирует данные'''
    
    for w in purpose.split():
        w = morph.parse(w)[0].normal_form
        if w == 'образование':
            return 'получение образования'
        if w == 'автомобиль':
            return 'операции с автомобилем'
        if w == 'свадьба':
            return 'проведение свадьбы'
    return 'операции с недвижимостью'

data['purpose_category'] = data['purpose'].apply(purpose_column) # применяем функцию к датафрейму и создаем столбец 

In [17]:
data.head(20) # проверка

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,операции с недвижимостью


### Изменение типов данных

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21470 entries, 0 to 21469
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21470 non-null  int64  
 1   days_employed     21470 non-null  float64
 2   dob_years         21470 non-null  int64  
 3   education         21470 non-null  object 
 4   education_id      21470 non-null  int64  
 5   family_status     21470 non-null  object 
 6   family_status_id  21470 non-null  int64  
 7   gender            21470 non-null  object 
 8   income_type       21470 non-null  object 
 9   debt              21470 non-null  int64  
 10  total_income      21470 non-null  float64
 11  purpose           21470 non-null  object 
 12  purpose_category  21470 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Тип `float` в ежемесячном доходе затруднит дальнейшую работу, преобразуем его в `int`. 

In [19]:
data['total_income'] = data['total_income'].astype('int')
data.info() # проверяем

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21470 entries, 0 to 21469
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21470 non-null  int64  
 1   days_employed     21470 non-null  float64
 2   dob_years         21470 non-null  int64  
 3   education         21470 non-null  object 
 4   education_id      21470 non-null  int64  
 5   family_status     21470 non-null  object 
 6   family_status_id  21470 non-null  int64  
 7   gender            21470 non-null  object 
 8   income_type       21470 non-null  object 
 9   debt              21470 non-null  int64  
 10  total_income      21470 non-null  int32  
 11  purpose           21470 non-null  object 
 12  purpose_category  21470 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(6)
memory usage: 2.0+ MB


### Категоризация дохода

In [20]:
def income_column(income):
    
    '''функция создает столбец с категориями доходов на основании диапозонов'''
    
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

data['total_income_category'] = data['total_income'].apply(income_column)  # применяем функцию к датафрейму и создаем столбец 


In [21]:
data.head(20) # проверка

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы,C
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,операции с недвижимостью,B
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операции с недвижимостью,B
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,получение образования,C
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,проведение свадьбы,C
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,операции с недвижимостью,C


### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

In [22]:
education_dict = data[['education', 'education_id']]
family_status_dict = data[['family_status','family_status_id']]
# создание словарей для дальнейшего обращения по идентификатору

In [23]:
data.drop(columns = ['education', 'family_status', 'total_income', 'purpose'],axis = 1, inplace=True)
# удаление 'лишних' столбцов

In [24]:
pd.options.display.max_rows = 22000 
data.head(200) # смотрим на итоговый датафрейм

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,purpose_category,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,операции с недвижимостью,B
1,1,4024.803754,36,1,0,F,сотрудник,0,операции с автомобилем,C
2,0,5623.422610,33,1,0,M,сотрудник,0,операции с недвижимостью,C
3,3,4124.747207,32,1,0,M,сотрудник,0,получение образования,B
4,0,340266.072047,53,1,1,F,пенсионер,0,проведение свадьбы,C
5,0,926.185831,27,0,1,M,компаньон,0,операции с недвижимостью,B
6,0,2879.202052,43,0,0,F,компаньон,0,операции с недвижимостью,B
7,0,152.779569,50,1,0,M,сотрудник,0,получение образования,C
8,2,6929.865299,35,0,1,F,сотрудник,0,проведение свадьбы,C
9,0,2188.756445,41,1,0,M,сотрудник,0,операции с недвижимостью,C


## Ответы на вопросы

### 1. Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [25]:
kids_debt = pd.DataFrame() # создаем новый датафрейм
kids_debt['clients with debt'] = data.groupby('children')['debt'].sum() # первый столбец - задолженности у клиентов с n детей
kids_debt['all clients'] = data.groupby('children')['debt'].count() # второй - всего клиентов с n детей
kids_debt['result'] = kids_debt['clients with debt'] / kids_debt['all clients'] # отношение задолженносте к кол-ву клиентов
kids_debt['percent'] = kids_debt['result'].apply(lambda x: format(x, '.2%')) # перевод в проценты
kids = kids_debt.sort_values(by='result') # сортировка по возрастанию
print(f'Подсчёт задолженностей по количеству детей:\n {kids}') # вывод датафрейма


Подсчёт задолженностей по количеству детей:
           clients with debt  all clients    result percent
children                                                  
5                         0            9  0.000000   0.00%
0                      1063        14106  0.075358   7.54%
3                        27          330  0.081818   8.18%
1                       445         4856  0.091639   9.16%
2                       202         2128  0.094925   9.49%
4                         4           41  0.097561   9.76%


По последним трем значениям видно, что чем больше детей в семье, тем выше процент задолженностей среди клиентов, при этом чаще всего в срок платят клиенты без детей. Клиентов с пятью детьми слишком мало, чтобы делать выводы.

### 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
fam_status_debt = pd.DataFrame()
fam_status_debt['clients with debt'] = data.groupby('family_status_id')['debt'].sum()
fam_status_debt['all clients'] = data.groupby('family_status_id')['debt'].count()
fam_status_debt['result'] = fam_status_debt['clients with debt'] / fam_status_debt['all clients']
fam_status_debt['percent'] = fam_status_debt['result'].apply(lambda x: format(x, '.2%'))
fam = fam_status_debt.sort_values(by='result')
print(f'Подсчёт задолженностей по семейному статусу:\n {fam}')


Подсчёт задолженностей по семейному статусу:
                   clients with debt  all clients    result percent
family_status_id                                                  
2                                63          959  0.065693   6.57%
3                                85         1195  0.071130   7.11%
0                               931        12344  0.075421   7.54%
1                               388         4162  0.093224   9.32%
4                               274         2810  0.097509   9.75%


In [27]:
family_status_dict.value_counts() # посмотрим значение статусов

family_status          family_status_id
женат / замужем        0                   12344
гражданский брак       1                    4162
Не женат / не замужем  4                    2810
в разводе              3                    1195
вдовец / вдова         2                     959
dtype: int64

Меньше всего задолженностей у тех, кто состоял, либо состоит в браке. Не состоящие в браке клиенты чаще не платят в срок.

### 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
income_debt = pd.DataFrame()
income_debt['clients with debt'] = data.groupby('total_income_category')['debt'].sum()
income_debt['all clients'] = data.groupby('total_income_category')['debt'].count()
income_debt['result'] = income_debt['clients with debt'] / income_debt['all clients']
income_debt['percent'] = income_debt['result'].apply(lambda x: format(x, '.2%'))
income = income_debt.sort_values(by='result')
print(f'Подсчёт задолженностей по уровню дохода:\n {income}')

Подсчёт задолженностей по уровню дохода:
                        clients with debt  all clients    result percent
total_income_category                                                  
D                                     21          350  0.060000   6.00%
B                                    356         5041  0.070621   7.06%
A                                      2           25  0.080000   8.00%
C                                   1360        16032  0.084830   8.48%
E                                      2           22  0.090909   9.09%


Зависимости от уровня дохода нет. По нашим данным больше всего должников с уровнем дохода меньше 30000, при этом отличие от категории "А" всего один процент. Чаще платят в срок клиенты с уровнем дохода в диапозоне 30000 - 50000, разница с минимальным значением равна 3 процентам.

### 4. Как разные цели кредита влияют на его возврат в срок?

In [29]:
purpose_debt = pd.DataFrame()
purpose_debt['clients with debt'] = data.groupby('purpose_category')['debt'].sum()
purpose_debt['all clients'] = data.groupby('purpose_category')['debt'].count()
purpose_debt['result'] = purpose_debt['clients with debt'] / purpose_debt['all clients']
purpose_debt['percent'] = purpose_debt['result'].apply(lambda x: format(x, '.2%'))
purpose = purpose_debt.sort_values(by='result')
print(f'Подсчёт задолженностей по цели кредита:\n {purpose}')

Подсчёт задолженностей по цели кредита:
                           clients with debt  all clients    result percent
purpose_category                                                          
операции с недвижимостью                782        10814  0.072314   7.23%
проведение свадьбы                      186         2334  0.079692   7.97%
получение образования                   370         4014  0.092177   9.22%
операции с автомобилем                  403         4308  0.093547   9.35%


Видна зависимость от цели кредита. Клиенты, берущие кредит на недвижимость и свадьбу платят в срок чаще тех, кто хочет получить образование или приобрести автомобиль.

## Общий вывод

Выявлена зависимость оплаты кредита в срок от семейного статуса. Люди, которые не состояли в браке, вероятнее всего совершат несвоевременую оплату. При этом клиенты, состоящие в браке и без детей чаще становятся пунктуальными плательщиками.

Больше всего задолженностей выявлено у клиентов, взявших кредит на автомобиль, меньше всего у тех, целью кредита которых была недвижимость.

Уровень дохода влияет на платежи меньше всего.